# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=PollingLiquidator.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 PollingLiquidator

This notebook implements a simple polling approach to a liquidator. It loops, processes all possible liquidations, then sleeps.

In [ ]:
import datetime
import logging
import time
import typing

from AccountLiquidator import AccountLiquidator
from BaseModel import Group, LiquidationEvent, MarginAccount, MarginAccountMetadata, OpenOrders, TokenValue
from Context import Context
from Observables import EventSource
from Wallet import Wallet
from WalletBalancer import WalletBalancer


# PollingLiquidator class

In [Liquidation](Liquidation.ipynb) it says these are probably roughly the steps to run a liquidator:

1. Find all liquidatable margin accounts.
2. Pick the most appropriate of these margin accounts, based on that account's collatoralisation and the liquidator's token balances.
3. Pick the market with the most value in the margin account's openorders accounts.
4. Force cancellation of all outstanding orders for the margin account in that market.
5. Pick the market with the highest borrows and lowest deposits for the account being liquidated.
6. Build and send the PartialLiquidate instruction.
7. Convert the received tokens to your desired tokens.
8. Repeat from step 2 (if necessary) with fresh tokens.

The `PollingLiquidator` class performs steps 1, 2, 6, and 8. Steps 3, 4, and 5 are handled implicitly by the `AccountLiquidator` (in our case the `ForceCancelOrdersAccountLiquidator`). Step 7 is handled by the `Balancer`.

That's not quite the whole story though.

We want to be quick to react to potential liquidation events, so as an optimisation this class:
* Pulls down all margin accounts
* Selects all those accounts which have borrows and currently have a collateral ratio less than the Group's 'initial collateral ratio' - ripe 🥭 mangoes
* Polls for Group and price changes, and processes possible liquidations on those ripe 🥭 accounts.

The reason for this is: pulling down and processing all margin accounts is processor and bandwidth intensive and takes a long time. The ripe 🥭 accounts are also less likely to change - liquidation events are triggered by price changes, not trades.

Processing only the ripe 🥭 accounts in a tighter loop makes it more likely we'll come across a liquidation early enough to be able to liquidate it.

This does also increase the potential of liquidating non-liquidatable accounts. For example, if we use cached data on a margin account's borrows and the user manually reduces those borrows, the `PollingLiquidator` could still try to liquidate it based on the outdated borrows. This may lead to an increase in `MangoErrorCode::NotLiquidatable` errors in the logs, but these typically aren't a problem.

In [ ]:
class PollingLiquidator:
    def __init__(self, context: Context, wallet: Wallet, account_liquidator: AccountLiquidator, wallet_balancer: WalletBalancer, worthwhile_threshold: float = 0.01, throttle_to_seconds: int = 5, ripe_iterations: int = 10):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.context: Context = context
        self.wallet: Wallet = wallet
        self.account_liquidator: AccountLiquidator = account_liquidator
        self.wallet_balancer: WalletBalancer = wallet_balancer
        self.worthwhile_threshold: float = worthwhile_threshold
        self.throttle_to_seconds: int = throttle_to_seconds
        self.ripe_iterations: int = ripe_iterations
        self.liquidations: EventSource[LiquidationEvent] = EventSource[LiquidationEvent]()

    def run(self):
        self.logger.info("Fetching all margin accounts...")
        group = Group.load(self.context)
        prices = group.fetch_token_prices()
        margin_accounts = MarginAccount.load_all_for_group(self.context, self.context.program_id, group)
        open_orders = OpenOrders.load_raw_open_orders_account_infos(self.context, group)
        for margin_account in margin_accounts:
            margin_account.install_open_orders_accounts(group, open_orders)
        self.logger.info(f"Fetched {len(margin_accounts)} margin accounts to process.")
        nonzero: typing.List[MarginAccountMetadata] = []
        for margin_account in margin_accounts:
            balance_sheet = margin_account.get_balance_sheet_totals(group, prices)
            if balance_sheet.collateral_ratio > 0:
                balances = margin_account.get_intrinsic_balances(group)
                nonzero += [MarginAccountMetadata(margin_account, balance_sheet, balances)]
        self.logger.info(f"Of those {len(margin_accounts)}, {len(nonzero)} have a nonzero collateral ratio.")

        ripe_metadata = filter(lambda mam: mam.balance_sheet.collateral_ratio <= group.init_coll_ratio, nonzero)
        ripe = list(map(lambda mam: mam.margin_account, ripe_metadata))
        self.logger.info(f"Of those {len(nonzero)}, {len(ripe)} are ripe 🥭.")

        for counter in range(self.ripe_iterations):
            self.logger.info(f"Update {counter} of {self.ripe_iterations} - {len(ripe)} ripe 🥭 accounts.")
            started_at = time.time()

            group = Group.load(self.context)
            prices = group.fetch_token_prices()
            updated: typing.List[MarginAccountMetadata] = []
            for margin_account in ripe:
                balance_sheet = margin_account.get_balance_sheet_totals(group, prices)
                balances = margin_account.get_intrinsic_balances(group)
                updated += [MarginAccountMetadata(margin_account, balance_sheet, balances)]

            liquidatable = list(filter(lambda mam: mam.balance_sheet.collateral_ratio <= group.maint_coll_ratio, updated))
            self.logger.info(f"Of those {len(updated)}, {len(liquidatable)} are liquidatable.")

            above_water = list(filter(lambda mam: mam.collateral_ratio > 1, liquidatable))
            self.logger.info(f"Of those {len(liquidatable)} liquidatable margin accounts, {len(above_water)} are 'above water' margin accounts with assets greater than their liabilities.")

            worthwhile = list(filter(lambda mam: mam.assets - mam.liabilities > self.worthwhile_threshold, above_water))
            self.logger.info(f"Of those {len(above_water)} above water margin accounts, {len(worthwhile)} are worthwhile margin accounts with more than ${self.worthwhile_threshold} net assets.")

            highest_first = sorted(worthwhile, key=lambda mam: mam.assets - mam.liabilities, reverse=True)
            for mam in highest_first:
                balances_before = group.fetch_balances(self.wallet.address)
                self.logger.info("Wallet balances before:")
                TokenValue.report(self.logger.info, balances_before)

                self.logger.info(f"Margin account balances before:\n{mam.balances}")
                self.logger.info(f"Liquidating margin account: {mam.margin_account}\n{mam.balance_sheet}")
                transaction_id = self.account_liquidator.liquidate(group, mam.margin_account, prices)
                if transaction_id is None:
                    self.logger.info("No transaction sent.")
                else:
                    self.logger.info(f"Transaction ID: {transaction_id} - waiting for confirmation...")

                    self.context.wait_for_confirmation(transaction_id)

                    group_after = Group.load(self.context)
                    margin_account_after_liquidation = MarginAccount.load(self.context, mam.margin_account.address, group_after)
                    intrinsic_balances_after = margin_account_after_liquidation.get_intrinsic_balances(group_after)
                    self.logger.info(f"Margin account balances after: {intrinsic_balances_after}")

                    self.logger.info("Wallet Balances After:")
                    balances_after = group_after.fetch_balances(self.wallet.address)
                    TokenValue.report(self.logger.info, balances_after)

                    liquidation_event = LiquidationEvent(datetime.datetime.now(),
                                                         transaction_id,
                                                         self.wallet.address,
                                                         margin_account_after_liquidation.address,
                                                         balances_before,
                                                         balances_after)

                    self.logger.info("Wallet Balances Changes:")
                    changes = TokenValue.changes(balances_before, balances_after)
                    TokenValue.report(self.logger.info, changes)

                    self.liquidations.publish(liquidation_event)
                    self.wallet_balancer.balance(prices)

            time_taken = time.time() - started_at
            should_sleep_for = self.throttle_to_seconds - int(time_taken)
            sleep_for = max(should_sleep_for, 0)
            self.logger.info(f"Check of all ripe 🥭 accounts complete. Time taken: {time_taken:.2f} seconds, sleeping for {sleep_for} seconds...")
            time.sleep(sleep_for)


# 🏃 Running

In [ ]:
if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)

    from AccountLiquidator import NullAccountLiquidator
    from Context import default_context
    from Wallet import default_wallet
    from WalletBalancer import NullWalletBalancer

    if default_wallet is None:
        print("No default wallet file available.")
    else:
        liquidator = PollingLiquidator(default_context, default_wallet, NullAccountLiquidator(), NullWalletBalancer())
        liquidator.run()
